# Algorithm Graph Export

Demonstrates how a tensorflow graph for a deconvolution algorithm can be built and then exported for use from other clients.

In [1]:
from tfdecon import restoration
import tensorflow as tf
import tempfile
import shutil
import os

Initialize the graph by specifying parameters that are constant within it.  For example, all of the following are not dynamic parts of each graph and must be set ahead of time:

- Number of dimensions for input/output data (1D vs 2D vs 3D)
    - \* This is necessary because TF FFT functions are not dynamic
- Domain in which FFT will be run (spatial vs frequency)

In [2]:
ndims = 3
domain_type = 'complex'
algo = restoration.RichardsonLucyDeconvolver(
    ndims, pad_mode='log2', real_domain_fft=(domain_type == 'real')
).initialize()

Set export directory into which the graph will be saved and clear it out, if it already exists:

In [3]:
export_dir = os.path.abspath(os.path.normpath('../../tensorflow/richardson-lucy-{}-{}d'.format(domain_type, ndims)))
export_dir

if os.path.exists(export_dir):
    shutil.rmtree(export_dir)

Write the graph out to a plain text "pbtxt" file:

In [29]:
algo.graph.save(export_dir, save_as_text=True)

INFO:tensorflow:No assets to save.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: b'C:\\Users\\User\\repos\\codex-analysis\\deconvolution\\tensorflow\\richardson-lucy-complex-2d\\saved_model.pbtxt'


As a test, reload the graph and make sure that operations within it are retrievable:

In [12]:
g = tf.Graph()
with tf.Session(graph=g) as sess:
    saver = tf.saved_model.loader.load(sess, [tf.saved_model.tag_constants.SERVING], export_dir) 

INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:The specified SavedModel has no variables; no checkpoints were restored.


In [9]:
import numpy as np
with tf.Session(graph=g) as sess:
    dh = g.get_tensor_by_name("data:0")
    kh = g.get_tensor_by_name("kernel:0")
    ph = g.get_tensor_by_name("padding_mode:0")
    ih = g.get_tensor_by_name("niter:0")
    o = g.get_tensor_by_name("result:0")
    res = sess.run(o, feed_dict={dh: np.ones((5, 5, 5)), kh: np.ones((5, 5, 5)), ph: 'log2', ih: 10})

In [10]:
res

array([[[7.21188087e-09, 1.52314598e-07, 2.65712642e-05, 4.98863346e-06,
         7.41669396e-07],
        [1.52314556e-07, 3.21687867e-06, 5.61184075e-04, 1.05359824e-04,
         1.56640272e-05],
        [2.65712661e-05, 5.61184366e-04, 9.78985354e-02, 1.83800161e-02,
         2.73258844e-03],
        [4.98863574e-06, 1.05359926e-04, 1.83800068e-02, 3.45076714e-03,
         5.13031555e-04],
        [7.41669453e-07, 1.56640453e-05, 2.73258938e-03, 5.13031904e-04,
         7.62732889e-05]],

       [[1.52314598e-07, 3.21688094e-06, 5.61183842e-04, 1.05359839e-04,
         1.56640326e-05],
        [3.21688049e-06, 6.79403965e-05, 1.18521927e-02, 2.22519645e-03,
         3.30823910e-04],
        [5.61184715e-04, 1.18522001e-02, 2.06761408e+00, 3.88185471e-01,
         5.77122346e-02],
        [1.05359955e-04, 2.22519878e-03, 3.88185471e-01, 7.28800893e-02,
         1.08352154e-02],
        [1.56640472e-05, 3.30824318e-04, 5.77122159e-02, 1.08352182e-02,
         1.61089061e-03]],

      